In [2]:
import os
import pandas as pd
from IPython.display import display

ROOT_DIR = os.environ.get('PYTHONPATH')

## **Load Entire VEP Output into Database**

In [3]:
VEP_OUTPUT_DIR = os.path.join(ROOT_DIR, 'files/vep_troubleshoot/uniprot_human_variants_vep_all.csv')

VEP_DF = pd.read_csv(VEP_OUTPUT_DIR, sep='\t')
display(VEP_DF)

/var/folders/y3/mxnsyg_910x11q_t3z54wkbr0000gn/T/ipykernel_8609/635972828.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  VEP_DF = pd.read_csv(VEP_OUTPUT_DIR, sep='\t')


,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
0,NC_000002.12:g.88978974G>A,A0A075B6H7.48,G,A,missense_variant,4,P/L,-,-,YES
1,NC_000002.12:g.88978975G>T,A0A075B6H7.48,G,T,missense_variant,4,P/T,-,-,YES
2,NC_000002.12:g.88978971G>A,A0A075B6H7.48,G,A,missense_variant,5,A/V,-,2.033e-05,YES
3,NC_000002.12:g.88978972C>T,A0A075B6H7.48,C,T,missense_variant,5,A/T,-,6.774e-06,YES
4,NC_000002.12:g.88978968T>G,A0A075B6H7.48,T,G,missense_variant,6,Q/P,-,6.773e-06,YES
...,...,...,...,...,...,...,...,...,...,...
10681953,NC_000001.11:g.158999996T>C,W6CW81.33,T,C,missense_variant,106,H/R,-,2.63e-05,YES
10681954,NC_000001.11:g.158999987G>C,W6CW81.33,G,C,missense_variant,109,S/C,-,-,YES
10681955,NC_000001.11:g.158999987G>T,W6CW81.33,G,T,missense_variant,109,S/Y,-,1.314e-05,YES
10681956,NC_000001.11:g.158999978G>A,W6CW81.33,G,A,missense_variant,112,A/V,-,-,YES


## **See How Many If Remove Redundancy AND Missense Only**

In [11]:

import re

encountered_dict = {}

def generate_key(tup):
   return getattr(tup, 'SWISSPROT') + str(getattr(tup, 'Protein_position')) + getattr(tup, 'Amino_acids')

boolMask = []
missenseRegex = r'(missense_variant)'

for row in VEP_DF.itertuples():
   key = generate_key(row)
   consequence = getattr(row, 'Consequence')
   try:
      if encountered_dict[key] == True:
         boolMask.append(False)
   except KeyError:
      encountered_dict[key] = True
      if bool(re.search(consequence, missenseRegex)) == True:
         boolMask.append(True)
      else:
         boolMask.append(False)

VEP_NR = VEP_DF[boolMask]

display(VEP_NR)

,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
0,NC_000002.12:g.88978974G>A,A0A075B6H7.48,G,A,missense_variant,4,P/L,-,-,YES
1,NC_000002.12:g.88978975G>T,A0A075B6H7.48,G,T,missense_variant,4,P/T,-,-,YES
2,NC_000002.12:g.88978971G>A,A0A075B6H7.48,G,A,missense_variant,5,A/V,-,2.033e-05,YES
3,NC_000002.12:g.88978972C>T,A0A075B6H7.48,C,T,missense_variant,5,A/T,-,6.774e-06,YES
4,NC_000002.12:g.88978968T>G,A0A075B6H7.48,T,G,missense_variant,6,Q/P,-,6.773e-06,YES
...,...,...,...,...,...,...,...,...,...,...
10681953,NC_000001.11:g.158999996T>C,W6CW81.33,T,C,missense_variant,106,H/R,-,2.63e-05,YES
10681954,NC_000001.11:g.158999987G>C,W6CW81.33,G,C,missense_variant,109,S/C,-,-,YES
10681955,NC_000001.11:g.158999987G>T,W6CW81.33,G,T,missense_variant,109,S/Y,-,1.314e-05,YES
10681956,NC_000001.11:g.158999978G>A,W6CW81.33,G,A,missense_variant,112,A/V,-,-,YES


## **See How Many Rows Have gnomAD_AF**

In [35]:
with_gnomAD_df = VEP_DF[VEP_DF['gnomADg_AF'] != '-']

display(with_gnomAD_df)

output_path = os.path.join(ROOT_DIR, 'files/vep_troubleshoot/vep_with_gnomAD_AF.csv')

with_gnomAD_df.to_csv(output_path, sep='\t', index=False)

gnomAD_common_mask = []

for row in with_gnomAD_df.itertuples():
   
   af = float(getattr(row, 'gnomADg_AF'))
   
   if af >= 0.01:
      gnomAD_common_mask.append(True)
   else:
      gnomAD_common_mask.append(False)
      
gnomAD_common_df = with_gnomAD_df[gnomAD_common_mask]

display(gnomAD_common_df)

,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
2,NC_000002.12:g.88978971G>A,A0A075B6H7.48,G,A,missense_variant,5,A/V,-,2.033e-05,YES
3,NC_000002.12:g.88978972C>T,A0A075B6H7.48,C,T,missense_variant,5,A/T,-,6.774e-06,YES
4,NC_000002.12:g.88978968T>G,A0A075B6H7.48,T,G,missense_variant,6,Q/P,-,6.773e-06,YES
6,NC_000002.12:g.88978966G>A,A0A075B6H7.48,G,A,missense_variant,7,L/F,-,6.771e-06,YES
17,NC_000002.12:g.88978763G>A,A0A075B6H7.48,G,A,missense_variant,18,T/I,-,6.757e-06,YES
...,...,...,...,...,...,...,...,...,...,...
10681948,NC_000001.11:g.159000012G>C,W6CW81.33,G,C,missense_variant,101,L/V,-,6.574e-06,YES
10681949,NC_000001.11:g.159000008C>G,W6CW81.33,C,G,missense_variant,102,G/A,-,6.573e-06,YES
10681953,NC_000001.11:g.158999996T>C,W6CW81.33,T,C,missense_variant,106,H/R,-,2.63e-05,YES
10681955,NC_000001.11:g.158999987G>T,W6CW81.33,G,T,missense_variant,109,S/Y,-,1.314e-05,YES


,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
59,NC_000002.12:g.88978700A>G,A0A075B6H7.48,A,G,missense_variant,39,V/A,-,0.0208,YES
95,NC_000002.12:g.88978653T>A,A0A075B6H7.48,T,A,missense_variant,55,T/S,-,0.0113,YES
143,NC_000002.12:g.88978584T>C,A0A075B6H7.48,T,C,missense_variant,78,S/G,-,0.2179,YES
411,NC_000002.12:g.90190408C>T,A0A075B6H8.46,C,T,missense_variant,27,T/I,-,0.7535,YES
605,NC_000022.11:g.22201720G>C,A0A075B6I3.37,G,C,"missense_variant,splice_region_variant",16,G/R,-,0.01368,YES
...,...,...,...,...,...,...,...,...,...,...
10681493,NC_000008.11:g.74029040C>T,Q9Y6Y9.185,C,T,missense_variant,157,P/S,-,0.01263,YES
10681854,NC_000019.10:g.51704275G>A,W5XKT8.43,G,A,missense_variant,246,G/S,-,0.02386,YES
10681877,NC_000019.10:g.51704332T>A,W5XKT8.43,T,A,missense_variant,265,W/R,-,0.01797,YES
10681921,NC_000001.11:g.159000194A>T,W6CW81.33,A,T,missense_variant,40,L/Q,-,0.01915,YES


## **See How Many Rows Are Benign**
ambiguous annotation not removed

In [34]:
benignRegex = r'(benign)'

benignMask = []

def to_float(numStr):
   
   try:
      return float(numStr)
   except ValueError:
      return -1
   
for row in VEP_NR.itertuples():
   
   CLIN_SIG = getattr(row, 'CLIN_SIG')
   AF = to_float(getattr(row, 'gnomADg_AF'))
   
   isBenign = bool(re.search(benignRegex, CLIN_SIG))
   
   if isBenign:
      benignMask.append(True)
   else:
      benignMask.append(False)

VEP_BENIGN_DF = VEP_NR[benignMask]

display(VEP_BENIGN_DF)



,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
12190,NC_000002.12:g.210023010T>C,A0AUZ9.110,T,C,missense_variant,968,D/G,benign,0.0006836,YES
15535,NC_000011.10:g.19234818A>G,A0AVK6.137,A,G,missense_variant,231,I/T,likely_benign,0.0001445,YES
15623,NC_000011.10:g.19230828C>G,A0AVK6.137,C,G,missense_variant,358,S/T,benign,0.0122,YES
15853,NC_000011.10:g.19225738T>C,A0AVK6.137,T,C,missense_variant,674,I/V,benign,0.01515,YES
16001,NC_000011.10:g.19224728T>C,A0AVK6.137,T,C,missense_variant,845,N/S,benign,0.006511,YES
...,...,...,...,...,...,...,...,...,...,...
10680504,NC_000001.11:g.7745952C>T,Q9Y6Y1.171,C,T,missense_variant,1493,A/V,likely_benign,6.571e-05,YES
10680652,NC_000010.11:g.119892915G>A,Q9Y6Y8.175,G,A,missense_variant,45,A/T,benign,0.009739,YES
10680782,NC_000010.11:g.119898888C>T,Q9Y6Y8.175,C,T,missense_variant,209,P/S,benign,0.008413,YES
10681168,NC_000010.11:g.119926067C>T,Q9Y6Y8.175,C,T,missense_variant,718,A/V,benign,0.002959,YES


## **Remove Ambiguous Annotations from Benign Set**

In [25]:
ambiguousRegex = r'(uncertain|conflicting)'

benignUnambMask = []

for row in VEP_BENIGN_DF.itertuples():
   
   CLIN_SIG = getattr(row, 'CLIN_SIG')
   
   if bool(re.search(ambiguousRegex, CLIN_SIG)) == True:
      benignUnambMask.append(False)
   else:
      benignUnambMask.append(True)

BENIGN_UNAMB_DF = VEP_BENIGN_DF[benignUnambMask].reset_index().drop(columns=['index'])

display(BENIGN_UNAMB_DF)

,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
0,NC_000002.12:g.88978700A>G,A0A075B6H7.48,A,G,missense_variant,39,V/A,-,0.0208,YES
1,NC_000002.12:g.88978653T>A,A0A075B6H7.48,T,A,missense_variant,55,T/S,-,0.0113,YES
2,NC_000002.12:g.88978584T>C,A0A075B6H7.48,T,C,missense_variant,78,S/G,-,0.2179,YES
3,NC_000002.12:g.90190408C>T,A0A075B6H8.46,C,T,missense_variant,27,T/I,-,0.7535,YES
4,NC_000022.11:g.22201974T>C,A0A075B6I3.37,T,C,missense_variant,61,L/P,-,0.4648,YES
...,...,...,...,...,...,...,...,...,...,...
58913,NC_000008.11:g.74029040C>T,Q9Y6Y9.185,C,T,missense_variant,157,P/S,-,0.01263,YES
58914,NC_000008.11:g.119105892C>T,Q9Y6Z7.155,C,T,missense_variant,179,R/W,likely_benign,0.001565,YES
58915,NC_000019.10:g.51704275G>A,W5XKT8.43,G,A,missense_variant,246,G/S,-,0.02386,YES
58916,NC_000001.11:g.159000194A>T,W6CW81.33,A,T,missense_variant,40,L/Q,-,0.01915,YES


## **Remove Unreviewed Proteins**

In [28]:
REV_PROTEINS_PATH = os.path.join(ROOT_DIR, 'files/uniprot_human_reviewed.csv')

REV_PROTEINS_DF = pd.read_csv(REV_PROTEINS_PATH, sep='\t')

UNIPROT_DICT = {}

for row in REV_PROTEINS_DF.itertuples():
   
   UNIPROT_DICT[getattr(row, 'Entry')] = True

reviewedMask = []

for row in BENIGN_UNAMB_DF.itertuples():
   
   uniprot = re.sub(r'\..*','',getattr(row, 'SWISSPROT'))
   
   try:
      if UNIPROT_DICT[uniprot] == True:
         reviewedMask.append(True)
   except KeyError:
      reviewedMask.append(False)

REV_BENIGN_DF = BENIGN_UNAMB_DF[reviewedMask]

display(REV_BENIGN_DF)
   

,#Uploaded_variation,SWISSPROT,REF_ALLELE,Allele,Consequence,Protein_position,Amino_acids,CLIN_SIG,gnomADg_AF,CANONICAL
0,NC_000002.12:g.88978700A>G,A0A075B6H7.48,A,G,missense_variant,39,V/A,-,0.0208,YES
1,NC_000002.12:g.88978653T>A,A0A075B6H7.48,T,A,missense_variant,55,T/S,-,0.0113,YES
2,NC_000002.12:g.88978584T>C,A0A075B6H7.48,T,C,missense_variant,78,S/G,-,0.2179,YES
3,NC_000002.12:g.90190408C>T,A0A075B6H8.46,C,T,missense_variant,27,T/I,-,0.7535,YES
4,NC_000022.11:g.22201974T>C,A0A075B6I3.37,T,C,missense_variant,61,L/P,-,0.4648,YES
...,...,...,...,...,...,...,...,...,...,...
58913,NC_000008.11:g.74029040C>T,Q9Y6Y9.185,C,T,missense_variant,157,P/S,-,0.01263,YES
58914,NC_000008.11:g.119105892C>T,Q9Y6Z7.155,C,T,missense_variant,179,R/W,likely_benign,0.001565,YES
58915,NC_000019.10:g.51704275G>A,W5XKT8.43,G,A,missense_variant,246,G/S,-,0.02386,YES
58916,NC_000001.11:g.159000194A>T,W6CW81.33,A,T,missense_variant,40,L/Q,-,0.01915,YES


## **Check with `vep_benign_variants_with_clusters_nr.csv`**

In [22]:
OTHER_BENIGN_FILE = os.path.join(ROOT_DIR, 'synced_files/vep_variants/vep_benign_variants_with_clusters_nr.csv')

OTHER_BENIGN_DF = pd.read_csv(OTHER_BENIGN_FILE, sep='\t')

display(OTHER_BENIGN_DF)

,uniprot,position,WT,Mut,cluster,category
0,A0A075B6H7,39,V,A,D2HJ94,LB/B
1,A0A075B6H7,55,T,S,D2HJ94,LB/B
2,A0A075B6H7,78,S,G,D2HJ94,LB/B
3,A0A075B6H8,27,T,I,A0A0H4LVB5,LB/B
4,A0A075B6I3,16,G,R,G1PYU6,LB/B
...,...,...,...,...,...,...
59826,Q9Y6Z7,179,R,W,L9LCR2,LB/B
59827,W5XKT8,246,G,S,A0A2Y9FYN0,LB/B
59828,W5XKT8,265,W,R,A0A2Y9FYN0,LB/B
59829,W6CW81,40,L,Q,A0A2K6E921,LB/B
